<a href="https://colab.research.google.com/github/DVPhong/AI_Project/blob/main/Title%20generator%20with%20LSTM/Title_generator_with_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import string
import json

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
# !pip install unrar
# !unrar x '/content/drive/MyDrive/Colab Notebooks/title generator/Data.rar'

In [4]:
# df_US = pd.read_csv('/content/Data/USvideos.csv')
# df_CA = pd.read_csv('/content/Data/CAvideos.csv')
# df_GB = pd.read_csv('/content/Data/GBvideos.csv')

# data_US = json.load(open('/content/Data/US_category_id.json'))
# data_CA = json.load(open('/content/Data/CA_category_id.json'))
# data_GB = json.load(open('/content/Data/GB_category_id.json'))

In [5]:
# df_US.head()

In [6]:
# create diction (id : title) from json file
# def category_extractor(data):
#   ids = [data['items'][i].get('id') for i in range(len(data['items']))]
#   titles = [data['items'][i]['snippet'].get('title') for i in range(len(data['items']))]

#   ids = list(map(int, ids))
#   diction = dict()
#   for id, title in zip(ids, titles):
#     diction[id] = title
#   return diction

In [7]:
#create new column category_title by mapping id
# df_US['category_title'] = df_US['category_id'].map(category_extractor(data_US))
# df_CA['category_title'] = df_CA['category_id'].map(category_extractor(data_CA))
# df_GB['category_title'] = df_GB['category_id'].map(category_extractor(data_GB))

# df = pd.concat([df_US, df_CA, df_GB], axis = 0, ignore_index=True)

# df = df.drop_duplicates('video_id') # drop rows with same video_id

# df.info()

In [8]:
# df.to_csv('/content/drive/MyDrive/Colab Notebooks/title generator/merge_data.csv', index = False)

In [9]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/title generator/merge_data.csv')
df.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,category_title
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22.0,2017-11-13T17:13:01.000Z,SHANtell martin,748374.0,57527.0,2966.0,15954.0,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...,People & Blogs
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24.0,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783.0,97185.0,6146.0,12703.0,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John...",Entertainment
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23.0,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434.0,146033.0,5339.0,8181.0,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...,Comedy
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24.0,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168.0,10172.0,666.0,2146.0,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...,Entertainment
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24.0,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731.0,132235.0,1989.0,17518.0,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...,Entertainment


In [28]:
import re
import string

def process_text(text):
  text = str(text) #some values are float

  text = re.sub(r'[%s]' % re.escape(string.punctuation), '',text) #remove punctuation

  text = re.sub(r'\s+', ' ',text) #remove duplicate space
  text = text.lower()

  text = text.encode('utf-8').decode('ascii', errors='ignore')

  return text

In [29]:
all_titles = df['title'].to_list()
corpus = [process_text(t) for t in all_titles]
corpus[:4]

['we want to talk about our marriage',
 'the trump presidency last week tonight with john oliver hbo',
 'racist superman rudy mancuso king bach lele pons',
 'nickelback lyrics real or fake']

In [27]:
s = set()
for tit in all_titles:
  s.add(type(tit))
print(s)

{<class 'float'>, <class 'str'>}


In [30]:
import keras
tokenizer = keras.preprocessing.text.Tokenizer()

In [31]:
def get_Ngram(corpus):
  tokenizer.fit_on_texts(corpus)
  total_words = len(tokenizer.word_index) + 1

  input_seq = []
  for title in corpus:
    token_list = tokenizer.texts_to_sequences([title])[0]
    for i in range(1, len(token_list)):
      input_seq.append(token_list[:i+1])

  return input_seq, total_words

input_seq, total_words = get_Ngram(corpus)

In [32]:
def pad_to_input(input_seq):
  max_len = max([len(seq) for seq in input_seq])
  input_sequences = np.array(keras.preprocessing.sequence.pad_sequences(input_seq,maxlen= max_len))

  preds, label = input_sequences[:, :-1], input_sequences[:, -1]
  label = keras.utils.to_categorical(label, num_classes=total_words)

  return preds, label, max_len

preds, label, max_len = pad_to_input(input_seq)
#preds is sequence Ngram, label is the next word of preds

In [33]:
print(input_seq[0:3])
print(np.array(label[0:3]).shape)
print(preds[0:3])
max_len

[[68, 513], [68, 513, 3], [68, 513, 3, 253]]
(3, 30156)
[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0  68]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0  68 513]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0  68 513   3]]


27

In [34]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dropout, Dense
from keras.losses import CategoricalCrossentropy
from keras.optimizers import Adam


def create_model(max_len, total_words):
  model = keras.models.Sequential([
    Embedding(input_dim = total_words,output_dim = 10, input_length = max_len-1),
    LSTM(30),
    Dropout(0.2),
    Dense(total_words,activation = 'softmax'),
  ])

  model.compile(
      loss = CategoricalCrossentropy(),
      optimizer = Adam()
  )

  return model

In [ ]:
model = create_model(max_len, total_words)
model.fit(x = preds, y = label,epochs = 10, verbose=5, batch_size=1)